# Language Modeling & Sentiment Analysis of IMDB movie reviews

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

In [2]:
bs=48
# bs=128

In [3]:
path = untar_data(URLs.IMDB)

## Language model

In [4]:
data_lm = (TextList.from_folder(path)
            .filter_by_folder(include=['train', 'test', 'unsup']) 
            .split_by_rand_pct(0.1, seed=42)
            .label_for_lm()           
            .databunch(bs=bs, num_workers=1))

len(data_lm.vocab.itos),len(data_lm.train_ds)

Process Process-30:
Process Process-22:
Process Process-28:
Process Process-32:
Process Process-17:
Process Process-18:
Process Process-26:
Process Process-27:
Process Process-19:
Process Process-20:
Process Process-23:
Process Process-25:
Process Process-29:
Traceback (most recent call last):
Process Process-31:
Traceback (most recent call last):
Process Process-24:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt

  File "/opt/conda/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/opt/conda/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/opt/conda/lib/python3.6/concurrent/futures/process.py", line 169, in _process_worker
    call_item = call_queue.get(block=True)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/opt/conda/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/opt/conda/lib/python3.6/concurrent/futures/process.py", line 169, in _process_worker
    call_item = call_queue.get(block=True)
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/opt/conda/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return sel

KeyboardInterrupt: 

In [20]:
data_lm.label_for_lm??

In [21]:
data_lm.label_const??

In [22]:
data_lm.label_from_func??

In [29]:
data_lm._label_from_list??

In [31]:
data_lm._label_list??

In [32]:
data_lm.save('lm_databunch')

In [4]:
data_lm = load_data(path, 'lm_databunch', bs=bs)

In [6]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=1.).to_fp16()

In [7]:
lr = 1e-2
lr *= bs/48

In [8]:
learn_lm.fit_one_cycle(1, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.436533,4.087692,0.287201,26:53


In [9]:
fit_one_cycle

In [10]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(10, lr/10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.081352,3.883096,0.309451,30:06


KeyboardInterrupt: 

In [11]:
learn_lm.save('fine_tuned_10')
learn_lm.save_encoder('fine_tuned_enc_10')

## Classifier

In [12]:
data_clas = (TextList.from_folder(path, vocab=data_lm.vocab)
             .split_by_folder(valid='test')
             .label_from_folder(classes=['neg', 'pos'])
             .databunch(bs=bs, num_workers=1))

In [13]:
data_clas.save('imdb_textlist_class')

In [5]:
data_clas = load_data(path, 'imdb_textlist_class', bs=bs, num_workers=1)

In [6]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5).to_fp16()
learn_c.load_encoder('fine_tuned_enc_10')
learn_c.freeze()

In [9]:
lr=2e-2
lr *= bs/48

In [17]:
learn_c.fit_one_cycle(1, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.292160,0.243463,0.899880,02:26


In [18]:
learn_c.save('1')

In [19]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(1, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.241684,0.185892,0.928720,02:44


In [20]:
learn_c.save('2nd')

In [7]:
learn_c.load('2nd')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (25000 items)
x: TextList
xxbos i was bored , around 10 pm , so i watched this movie . xxmaj and i could not stop laughing . xxmaj everything was so ridiculous . xxmaj the way the kids were acting like they were older than 11 just cracked me up . xxmaj one of the kids had a ring , that supposedly killed people after 3 or so years . xxmaj it gave me the impression that he wanted to be a gangster . 
 
  xxmaj it 's pretty hard to take little kids seriously , especially when it has to do with eating worms . xxmaj they act like everything is such a big deal , like if xxmaj billy ( the main character ) does n't eat the worms then the world will end . 
 
  xxmaj this is a good movie for little kids ( excluding the fact that a 5 year old says ' penis ' ) , but not for teens or adults who do n't want to waste their time .,xxbos xxmaj bill ( xxmaj buddy xxmaj rogers ) is sent to xxmaj new xxmaj york by his uncle ( xxmaj richard xxmaj tucker )

In [10]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(1, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.198448,0.165060,0.937800,04:22


In [11]:
learn_c.save('3rd')

In [12]:
learn_c.unfreeze()
learn_c.fit_one_cycle(2, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.169140,0.157897,0.939440,04:51
1,0.125248,0.159314,0.942160,05:23


In [39]:
learn_c.unfreeze()
learn_c.fit_one_cycle(2, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.120063,0.145701,0.947000,03:24
1,0.087303,0.152943,0.948080,03:09


In [13]:
learn_c.save('clas')

In [14]:
del learn_c

In [15]:
del data_lm

In [19]:
from __future__ import print_function  # for Python2
import sys

local_vars = list(locals().items())
for var, obj in local_vars:
    print(var, sys.getsizeof(obj))

__name__ 57
__doc__ 113
__package__ 16
__loader__ 16
__spec__ 16
__builtin__ 80
__builtins__ 80
_ih 264
_oh 240
_dh 72
In 264
Out 240
get_ipython 64
exit 56
quit 56
_ 56
__ 49
___ 49
_i 54
_ii 55
_iii 53
_i1 152
version 80
AUROC 1056
AccumulateScheduler 1056
AdamW 80
AdaptiveConcatPool2d 1056
AffineFunc 888
AffineMatrix 1056
AnnealFunc 888
Any 48
AnyStr 88
ArgStar 888
AverageMetric 1056
BCEFlat 136
BCEWithLogitsFlat 136
BatchNorm1dFlat 1056
BatchSampler 1056
BatchSamples 888
BnFreeze 1056
BoolOrTensor 80
BufferedWriter 400
ByteTensor 400
BytesIO 400
Callable 888
Callback 1056
CallbackHandler 1056
CallbackList 888
Category 1056
CategoryList 1056
CategoryProcessor 1056
ClassificationInterpretation 1056
Collection 888
Config 1056
ConfusionMatrix 1056
Counter 1056
CrossEntropyFlat 136
DataBunch 1056
DataFrame 1056
DataFrameOrChunks 80
DataLoader 1464
Dataset 1056
DatasetType 1056
Debugger 1056
DeviceDataLoader 1056
Dict 888
DistributedDataParallel 1056
DoubleTensor 400
EmptyLabel 1056
Empt

In [20]:
del learner

NameError: name 'learner' is not defined

In [21]:
torch.cuda.empty_cache()